<a href="https://colab.research.google.com/github/Julianfried/Fundamentos-de-datos/blob/main/Clase2_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![modelo](https://drive.google.com/uc?id=1xXfeu5ZlbEDrK_vo-vbsZ5Jv1TROgrsr)

In [1]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql CREATE SCHEMA IF NOT EXISTS humai

 * Starting PostgreSQL 12 database server
   ...done.
CREATE ROLE
 * postgresql+psycopg2://@/postgres


""


In [2]:
# extra dependencies
import pandas as pd
import sqlalchemy

engine = sqlalchemy.create_engine("postgresql+psycopg2://@/postgres")
conn = engine.connect()

akas = pd.read_csv("https://drive.google.com/uc?id=1MDgiAel34RZULi9a1WXuzGQOE9M5_aGp",nrows=200000,sep="\t")
akas.columns = [i.lower() for i in akas.columns]
crew = pd.read_csv("https://drive.google.com/uc?id=1vzlcrhm77MFw7ha_Xj3MyfPwhM7XhqW2",nrows=200000,sep="\t")
crew.columns = [i.lower() for i in crew.columns]
names = pd.read_csv("https://drive.google.com/uc?id=1Co0bwQQhmtMcTnZwZqWo_W1-nrFVgBFm",nrows=200000,sep="\t")
names.columns = [i.lower() for i in names.columns]
principals = pd.read_csv("https://drive.google.com/uc?id=1ojsyx3m-8-3x9LM_X_TS0EBN4JMrkyR4",nrows=200000,sep="\t")
principals.columns = [i.lower() for i in principals.columns]
ratings = pd.read_csv("https://drive.google.com/uc?id=1XCN67VhsVQmlr5BudNJGjHnqZR_DqetR",nrows=200000,sep="\t")
ratings.columns = [i.lower() for i in ratings.columns]
ratings.to_sql("ratings",engine,if_exists="append",method="multi",schema="humai",index=False)
del(ratings)
akas.to_sql("akas",engine,if_exists="append",method="multi",schema="humai",index=False)
del(akas)
crew.to_sql("crew",engine,if_exists="append",method="multi",schema="humai",index=False)
del(crew)
names.to_sql("names",engine,if_exists="append",method="multi",schema="humai",index=False)
del(names)
principals.to_sql("principals",engine,if_exists="append",method="multi",schema="humai",index=False)
del(principals)

## Más Ejercicios:

  1. Retornar el actor con mayor numero de apariciones en peliculas
  2. Retornar el puesto de trabajo menos común
  3. Devolver el titulo más largo de una pelicula incluyendo:
    - El rating
    - La cantidad de votos
  4. Crear una vista que contenga:
    - Las 10 peliculas con menor rating
    - Incluyendo titulo

In [17]:
# 1. Retornar el actor con mayor numero de apariciones en peliculas
%%sql
SELECT n.nconst as "Id", n.primaryName AS "Name", COUNT(tconst) AS "Apariciones"
FROM humai.principals p
INNER JOIN humai.names n
ON p.nconst = n.nconst
GROUP BY 2, 1
ORDER BY 3 DESC
;

 * postgresql+psycopg2://@/postgres


,Id,Name,Apariciones
0,nm0000428,D.W. Griffith,578
1,nm0005658,G.W. Bitzer,526
2,nm0001908,Gilbert M. 'Broncho Billy' Anderson,240
3,nm0199841,Bebe Daniels,217
4,nm0106218,John Randolph Bray,211
...,...,...,...
7004,nm0020697,Joseph Allen,1
7005,nm0139353,H.C. Carpenter,1
7006,nm0179915,Hazel Corker,1
7007,nm0047333,Joseph Bailey,1


In [26]:
# 2. Retornar el puesto de trabajo menos común
%%sql
SELECT category, COUNT(tconst) AS "Total puestos"
FROM humai.principals
GROUP BY category
ORDER BY COUNT(tconst)
;

 * postgresql+psycopg2://@/postgres


,category,Total puestos
0,archive_footage,73
1,production_designer,358
2,self,560
3,composer,2409
4,editor,2949
5,producer,6652
6,cinematographer,15386
7,director,24187
8,writer,33687
9,actress,40934


In [30]:
  #3. Devolver el titulo más largo de una pelicula incluyendo:
   # - El rating
   #- La cantidad de votos
%%sql
SELECT a.title AS "Titulo", r.averageRating AS "Raiting promedio", r.numVotes AS "Numero de votos", LENGTH(a.title) AS "Total de caracteres"
FROM humai.akas a
LEFT JOIN humai.ratings r
ON a.titleId = r.tconst
ORDER BY LENGTH(a.title) DESC
LIMIT 1;

 * postgresql+psycopg2://@/postgres


,Titulo,Raiting promedio,Numero de votos,Total de caracteres
0,"Winsor McCay, the Famous Cartoonist of the N.Y...",7.1,1687,138


In [48]:
# 4. Crear una vista que contenga:
#     Las 10 peliculas con menor rating
#     Incluyendo titulo

%%sql
CREATE OR REPLACE VIEW less_rating AS 
SELECT title, averagerating AS "Raiting"
FROM humai.akas a
INNER JOIN humai.ratings r
ON r.tconst = a.titleId
GROUP BY 1,2
ORDER BY r.averageRating
LIMIT 10;


 * postgresql+psycopg2://@/postgres


""


In [49]:
%sql SELECT * FROM less_rating

 * postgresql+psycopg2://@/postgres


,title,Raiting
0,Águilas de acero o los misterios de Tánger,1.0
1,Ett kärleksäventyr,1.0
2,Young Lochinvar,1.0
3,心の冒険,1.0
4,An Adventure in Hearts,1.0
5,Captain Dieppe,1.0
6,One Hour,1.1
7,The Falling of the Red Army,1.1
8,Kærlighedens Time,1.1
9,El derrumbamiento del Ejército Rojo,1.1
